# 본 페이지의 목표

-gensim에서 제공하는 fasttext 모델에 없는 단어를 학습시킨다.

-없는 단어를 학습하기 위해 크롤링을 하여 context를 모은다.

-크롤링한 context를 fasttext에 학습시킨다.

In [1]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import codecs
%matplotlib inline

In [42]:
list_items = pd.read_csv('C:/Users/admin/Desktop/NLP/description/skipgram/오타확인/total_typo_check_list.txt',header = None, delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
list_items = list_items[0]
print(list_items[0:3])

0    STABILISATION
1           MOOHAN
2            INLAY
Name: 0, dtype: object


# 기존에 존재하지 않은 단어 크롤링! 

크롤링을 통해 source,target description에 미포함된 단어의 context를 만든다.

만드는 규칙 따위 없다. 그냥 단어에 상관없이 문장을 다 더할것이다. 

In [90]:
list_items[212]

'INFOMED'

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re # 한글 분리
hangul = re.compile('[^가-힣]+')
english = re.compile('[^a-zA-Z0-9.,:-]+')
not_english = re.compile('[a-zA-Z0-9.,]+')
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument('--dns-prefetch-disable')
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14931")
#크롬 드라이버 경로를 위치시킨다
driver = webdriver.Chrome('C:/Users/admin/Desktop/NLP/chromedriver_v71/chromedriver', options=options)
# crawl_txt = []
driver.implicitly_wait(3)
# ex =['COFLEXINTERSPINOUSIMPLANTS']#list_items
for idx,search_item in enumerate(list_items[527:]):
    driver = webdriver.Chrome('C:/Users/admin/Desktop/NLP/chromedriver_v71/chromedriver', options=options)
    driver.implicitly_wait(3)
    url = "https://www.google.com"
    driver.get(url)
    elem = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input')
    elem.send_keys(str(search_item))
    elem.submit()
    driver.implicitly_wait(5) # 브라우저 검색 최소 지연시간 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    notices = soup.select('#fprsl > em')
    no_result = soup.select('#topstuff > div > div > p') 
    div_group=[]

    
    if(len(no_result)>0):
        print('검색결과 없음')
        continue
                             
    if(len(notices)>0): # 해당 단어의 예상 추천 단어 추천단어로 재검색 
        print('추천 단어로 재검색: ')
        driver = webdriver.Chrome('C:/Users/admin/Desktop/NLP/chromedriver_v71/chromedriver', options=options)
        driver.implicitly_wait(5)
        url = "https://www.google.com"
        driver.get(url)
        driver.implicitly_wait(5) # 브라우저 검색 최소 지연시간   
        elem = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input')
        elem.send_keys(str(notices[0].text))
        search_item = str(notices[0].text)
        elem.submit()
        driver.implicitly_wait(3) # 브라우저 검색 최소 지연시간    
    div_group=soup.find_all("span", {"class": "st"}) 
    for div in div_group: 
        try:
            a =str(div.get_text()).replace("...","")
#             temp2 = english.sub(' ',a ) #영어만 추출
            a_temp = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ_!』\\‘|\(\)\[\]\<\>`\'…》]', '', a)
            a_temp=re.sub('\W+','', a_temp)#두번에 걸쳐 특수문자를 제거 -> 그 후에도 남아있는 것은 영어가 아닌 단어라고 생각
            pattern = re.compile(r'\s+')
            a_temp = re.sub(pattern, '', a_temp)
            temp3 =not_english.sub('',a_temp)#영어가 아닌 단어 추출

            
            if len(temp3) > 0:#영어가 아닌 단어가 존재하면 넘어간다.
                continue
#             print(temp3)
            a = re.sub('[=+#/\?^;$.@*\"※~&%ㆍ_!』·“”\\‘|\(\)\[\]\<\>`\'…》]', '', a)
#             print(a)# 영어만 존재할 경우 append
            print('현재 크롤링된 index:%d , 현재 단어:%s '%(idx+527,str(search_item)))
            print('영어로된 크롤링 문장: ',a)
            crawl_txt.append(str(a))
                
        except NoSuchElementException:
                print("찾고자하는 검색결과가 하나도 없어서 에러남 ")
                
    driver.implicitly_wait(3)

In [ ]:
with codecs.open("C:/Users/admin/Desktop/NLP/description/skipgram/train_source_target_outdata.txt", 'w', "utf-8") as f:
    for data in crawl_txt:
        temp = str(data)+'\n'
        f.write(temp)

## fasttext 기존 모델을 불러와서 새로운 context학습시키기  
cc.en.300.bin은 https://github.com/facebookresearch/fastText/blob/master/docs/crawl-vectors.md 에서 제공한다.

In [1]:
#from gensim.models.fasttext import FastText 
from gensim.test.utils import datapath
from gensim.models.wrappers import FastText
cap_path = datapath('C:/Users/admin/Desktop/NLP/cc.en.300.bin')
fb_full = FastText.load_fasttext_format(cap_path)

In [3]:
print(len(fb_full.wv.vocab))

2000000


In [3]:
print(fb_full.wv['system'][:5])
print(len(fb_full.wv['system']))

[-0.06489275 -0.01497137  0.08029559  0.02019559 -0.03679956]
300


In [7]:
train_set_list=[]
with codecs.open("C:/Users/admin/Desktop/NLP/description/skipgram/train_source_target_outdata.txt", 'r', "utf-8") as f:
    train_set_list= f.readlines()
print(train_set_list[:5])

['Define stabilisation stabilisation synonyms, stabilisation pronunciation, stabilisation translation, English dictionary definition of stabilisation Noun 1 stabilisation\xa0\n', 'the act of stabilizing something or making it more stable\n', 'In recent years, there have been significant stabilisation interventions in places such as the Western Balkans, Haiti and Mali, among others The concept of\xa0\n', 'In-situ soil stabilisation is used throughout the building and civil engineering industries This is a quick and cost efficient method cheaper to modify existing soils\xa0\n', 'This note covers the stabilisation process, the statutory prohibitions against stabilisation and available safe harbours Free Practical Law trial To access this\xa0\n']


### (optional) List에 \xa0과 \n 제거
##### 사실상 제거하지 않아도 무방하다. 크롤링한 txt를 그냥 사용해도 된다.
##### 그냥 사용할 경우 새로운 word 학습 section으로 넘어가자.

In [10]:
train_set_list_=[]
for line in train_set_list:
    for char in ['\xa0', '\n']:
        line = line.replace(char,'')
    train_set_list_.append(line)
print(train_set_list_[:5])

['Define stabilisation stabilisation synonyms, stabilisation pronunciation, stabilisation translation, English dictionary definition of stabilisation Noun 1 stabilisation', 'the act of stabilizing something or making it more stable', 'In recent years, there have been significant stabilisation interventions in places such as the Western Balkans, Haiti and Mali, among others The concept of', 'In-situ soil stabilisation is used throughout the building and civil engineering industries This is a quick and cost efficient method cheaper to modify existing soils', 'This note covers the stabilisation process, the statutory prohibitions against stabilisation and available safe harbours Free Practical Law trial To access this']


In [11]:
print(len(train_set_list_))
print(len(train_set_list))

40766
40766


### 모델 새로운 word 학습  
gensim에서 제공하는 fastext로는 새로운 word를 추가할 수 없다. (여러가지 에러가 발생 )  
그래서  https://github.com/facebookresearch/fastText 를 설치하여 pretrained된 weight를 가지고 새로운 word를 추가해야한다.  
단 facebook research에서 제공하는 code는 linux 계열 or mac환경에서만 작동된다.  
설치 
> 1. git clone https://github.com/facebookresearch/fastText.git
> 2. cd fastText
> 3. make   
- - -
프로그램을 설치한 후 fastText directory (fastext root) 로 들어가서 아래의 명령어를 실행한다.  
> <pre><code>./fasttext skipgram  -input train_source_target_outdata.txt -output oov_train_model  -dim 300 -minCount 1 -verbose 2 -pretrainedVectors cc.en.300.vec</code></pre>
- - -
위 명령어의 의미는 https://github.com/facebookresearch/fastText.git 에서 자세한 내용은 확인할 수 있다.  
> ###### skipgram :  
>> cbow, skipgram, supervised가 있고 NLP 학습 모델을 무엇으로 할지 정하는 것.  
> ###### -input : 
>> 학습시킬 data를 받는 인자로 data는 문장으로 되어있다. out of vocabrary (oov) 를 학습하기 위해  
         크롤링을 한 data파일이 있을 것이다. 위에서 해당 문장을 train_source_target_outdata.txt 에 저장했고  
         해당 파일을 input으로 사용하였다.  
> ###### -output : 
>> 학습 후 나온 모델의 이름. 지정한 이름으로 두가지 파일인 .bin과 .vec 파일이 생성된다.
> ###### -dim : 
>> word embedding을 통해 나올 단어의 차원 수. pretrained weight가 300d 였으므로 같은 차원인 300으로 맞춰준다.  
> ###### -minCount : 
>> 최소 빈도 단어의 옵션으로 minCount보다 작은 빈도의 단어는 학습하지 않는다.  
> ###### -pretrainedVectors : 기존의 학습된 vector로 우린 cc.en.300.vec을 사용하여 model에 넣었다. 
- - -
해당 결과로 oov_train_model.bin과 oov_train_model.vec 파일이 생성된다.  

In [ ]:
from gensim.test.utils import datapath
from gensim.models.wrappers import FastText
fb_full2 = FastText.load_fasttext_format('oov_train_model.bin') 
print(len(fb_full2.wv.vocab)) # -> 2051129 의 vocab를 갖게된다. 

In [19]:
data = np.array([1,2,3])
for i in data:
    print(i)

1
2
3
